In [35]:
"""The pipes are arranged in a two-dimensional grid of tiles:

| is a vertical pipe connecting north and south.
- is a horizontal pipe connecting east and west.
L is a 90-degree bend connecting north and east.
J is a 90-degree bend connecting north and west.
7 is a 90-degree bend connecting south and west.
F is a 90-degree bend connecting south and east.
. is ground; there is no pipe in this tile.

S is the starting position; there is a pipe on this tile,
but the sketch doesn't show what shape the pipe has.
"""

example_1 =""".....
.F-7.
.|.|.
.L-J.
.....
"""

example_2 =""".....
.S-7.
.|.|.
.L-J.
.....
"""

example_2_1 ="""-L|F7
7S-7|
L|7||
-L-J|
L|-JF
"""

example_3 ="""..F7.
.FJ|.
SJ.L7
|F--J
LJ...
"""

example_3_1 ="""7-F7-
.FJ|7
SJLL7
|F--J
LJ.LJ
"""

In [1]:
import numpy as np
from collections import deque
from tqdm import tqdm

starting_pos = (2,0)
# Define the possible pipe types
pipe_types = {
    '|': [(-1, 0), (1, 0)],
    '-': [(0, -1), (0, 1)],
    'L': [(0, -1), (1, 0)],
    'J': [(0, 1), (1, 0)],
    '7': [(0, 1), (-1, 0)],
    'F': [(0, -1), (-1, 0)],
    '.': [],
    'S': [(0, 1), (0, -1), (1, 0), (-1, 0)]
}

def split_input(input_data):
    return [list(row) for row in input_data.splitlines()]

def verify_symbol(mirrored_connections, local_grid_array, x_row, x_col):
    """Verify that the connections are valid for the given pipe type"""
    # Check each position relative to 'X'
    for dr, dc in mirrored_connections:
        new_row, new_col = x_row + dr, x_col + dc
        if 0 <= new_row < local_grid_array.shape[0] and 0 <= new_col < local_grid_array.shape[1]:
            if (dr,dc) not in pipe_types[local_grid_array[new_row, new_col][0]]:
                return None
        else:
            return None
    return mirrored_connections

def check_pipe(x_row, x_col, local_grid_array, original_char, debug=False):
    """
    Determine if a pipe is valid based on a 3x3 grid.
    Valid pipes have exactly two connections and is the expected
    character, or 'S'
    """
    # Check each position relative to 'X'
    valid_pipes = 0
    is_starting_pos = False
    for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
        new_row, new_col = x_row + dr, x_col + dc
        if 0 <= new_row < local_grid_array.shape[0] and 0 <= new_col < local_grid_array.shape[1]:
            if debug:
                print(f"New row: {dr}, new col: {dc}, char: {local_grid_array[new_row, new_col]}")
            if (dr,dc) in pipe_types[local_grid_array[new_row, new_col][0]]:
                if debug:
                    print(f"Found valid pipe {local_grid_array[new_row, new_col]}")
                valid_pipes += 1
    # If there's exactly two valid pipes next to 'X', return True, else False
    if valid_pipes >= 2:
        if original_char != 'S':
            mirrored_connections = [(-dr, -dc) for dr, dc in pipe_types[original_char]]
            return verify_symbol(mirrored_connections, local_grid_array, x_row, x_col), is_starting_pos
        else:
            # Try all possible combinations of connections
            for guess_char, mirrored_connections in pipe_types.items():
                mirrored_connections = [(-dr, -dc) for dr, dc in mirrored_connections]
                result = verify_symbol(mirrored_connections, local_grid_array, x_row, x_col)
                if result:
                    print(f"Found starting position! Assuming 'S' is {guess_char}")
                    is_starting_pos = True
                    return result, is_starting_pos
    if debug:
        print(f"Invalid pipe at {x_row}, {x_col}, when checking: {original_char}, found {valid_pipes} valid pipes")
    return None, is_starting_pos

def shortest_path(pipes, start, end):
    # Create a queue for BFS
    queue = deque([(start, 0)])
    # Create a set to store visited nodes
    visited = set()
    while queue:
        node, steps = queue.popleft()
        # If this node is the destination, return the number of steps
        if node == end:
            return steps
        visited.add(node)
        for neighbor in pipes[node]:
            # If the neighbor has not been visited, enqueue it with the number of steps + 1
            if neighbor not in visited:
                queue.append((neighbor, steps + 1))
    # No path found
    return -1

with open('adventofcode.com_2023_day_10_input.txt', 'r') as f:
    input_string = f.read()

content = np.array(split_input(input_string))
display(content)
# Loop through every character, take a 3x3 grid around it, and check if it's a valid pipe
valid_pipes = {}
for row in range(content.shape[0]):
    for col in range(content.shape[1]):
        debug = False
        # Handle edge cases by creating partial grids
        grid_slice = content[max(0, row-1):min(content.shape[0], row+2), max(0, col-1):min(content.shape[1], col+2)].copy()
        # Determine the position of 'X' in the grid slice
        x_row, x_col = min(1, row), min(1, col)
        # Save the original character and mark the current position with 'X'
        original_char = grid_slice[x_row, x_col]
        if original_char == '-':
            debug = False
        grid_slice[x_row, x_col] = 'X'
        neighbors, found_start = check_pipe(x_row, x_col,grid_slice, original_char,debug)
        if neighbors:
            # print(f"Found valid pipe at {row}, {col}")
            valid_pipes[(row, col)] = [(row+dr, col+dc) for dr, dc in neighbors]
        if found_start:
            print(f"Setting starting position to {row}, {col}")
            starting_pos = (row, col)
# Update the grid with the new pipes, take shape  and mark pipes with emojis
new_grid = np.zeros(content.shape, dtype=str)
new_grid.fill('🚫')
for pos, neighbors in valid_pipes.items():
    new_grid[pos] = '🔗'
display(new_grid)
display(valid_pipes)
print(f"Found {len(valid_pipes)} valid pipes")
if starting_pos == (0,0):
    raise Exception("No starting position found!")
# run with tqdm to see progress
paths = []
for target in tqdm(valid_pipes.keys()):
    paths.append(shortest_path(valid_pipes, starting_pos, target))
# paths = [shortest_path(valid_pipes, starting_pos, target) for target in valid_pipes.keys()]
print(paths)
print(f"Max_length steps: {max(paths)}")

array([['.', '7', 'F', ..., '7', '.', 'F'],
       ['F', 'L', '7', ..., '.', 'F', '|'],
       ['7', '-', '7', ..., '-', 'F', '.'],
       ...,
       ['|', '7', '-', ..., 'L', '.', '.'],
       ['-', '|', 'F', ..., 'L', '-', '7'],
       ['L', 'J', 'J', ..., '|', 'J', 'J']], dtype='<U1')

Found starting position! Assuming 'S' is L
Setting starting position to 111, 14


array([['🚫', '🚫', '🚫', ..., '🚫', '🚫', '🚫'],
       ['🚫', '🔗', '🚫', ..., '🚫', '🚫', '🚫'],
       ['🚫', '🚫', '🔗', ..., '🚫', '🚫', '🚫'],
       ...,
       ['🚫', '🚫', '🚫', ..., '🚫', '🚫', '🚫'],
       ['🚫', '🔗', '🔗', ..., '🚫', '🔗', '🔗'],
       ['🚫', '🔗', '🚫', ..., '🚫', '🚫', '🚫']], dtype='<U1')

{(0, 3): [(0, 2), (1, 3)],
 (0, 6): [(0, 7), (0, 5)],
 (0, 12): [(0, 13), (1, 12)],
 (0, 37): [(0, 38), (1, 37)],
 (0, 41): [(0, 40), (1, 41)],
 (0, 50): [(0, 49), (1, 50)],
 (0, 54): [(0, 55), (1, 54)],
 (0, 55): [(0, 56), (0, 54)],
 (0, 56): [(0, 57), (0, 55)],
 (0, 57): [(0, 56), (1, 57)],
 (0, 67): [(0, 68), (0, 66)],
 (0, 68): [(0, 67), (1, 68)],
 (0, 72): [(0, 73), (0, 71)],
 (0, 80): [(0, 81), (0, 79)],
 (0, 81): [(0, 80), (1, 81)],
 (0, 83): [(0, 84), (1, 83)],
 (0, 85): [(0, 86), (1, 85)],
 (0, 86): [(0, 87), (0, 85)],
 (0, 107): [(0, 108), (0, 106)],
 (0, 115): [(0, 116), (1, 115)],
 (0, 116): [(0, 117), (0, 115)],
 (0, 117): [(0, 118), (0, 116)],
 (0, 120): [(0, 121), (0, 119)],
 (0, 121): [(0, 122), (0, 120)],
 (0, 129): [(0, 128), (1, 129)],
 (1, 1): [(1, 2), (0, 1)],
 (1, 14): [(2, 14), (0, 14)],
 (1, 25): [(2, 25), (0, 25)],
 (1, 27): [(1, 28), (2, 27)],
 (1, 28): [(1, 29), (1, 27)],
 (1, 29): [(1, 28), (0, 29)],
 (1, 36): [(1, 37), (0, 36)],
 (1, 37): [(1, 36), (0, 37)]

Found 15039 valid pipes


100%|██████████| 15039/15039 [02:26<00:00, 102.34it/s]

[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 5930, 5931, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 5902, 5903, 5906, 5907, 5910, 5911, 5928, 5929, 5932, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 4707, 4708, 4709, -1, 5901, 5904, 5905, 5908, 5909, 5912, 5927, 5926, 5933, 5934, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 3637, 3638, -1, -1, -1, -1, -1, 4706, 4705, 4710, 4713, 4714, 4793, 4794, 4795, 5900, 5